In [1]:
import requests
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import wikipedia
from PIL import Image
from io import BytesIO
from datetime import date
import xml.etree.ElementTree as ET


c:\Users\andre.de.souza.reis\Anaconda\envs\streamlit\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


#### Requesting all drivers names

In [8]:
# Request
data = requests.request("GET", 'http://ergast.com/api/f1/drivers?limit=10000').text

# Parsing to DataFrame
pilots = pd.read_xml(
    data,
    namespaces = {'doc': "http://ergast.com/mrd/1.5"},
    xpath = './/doc:Driver'
    )

# Joining first and last name
pilots = pilots.assign(Name = pilots[['GivenName', 'FamilyName']].apply(' '.join, axis = 1)).drop(['GivenName', 'FamilyName'], axis = 1)

# Getting simillar drivers
# simmilar = process.extractBests("Lewis", pilots['Name'].to_list(), limit = 10, score_cutoff=80)

# a = []
# for i in simmilar:
#     a.append(i[0])
# tuple(a)

#### Getting wikipedia picture

In [9]:
# text = requests.request("GET", 'http://en.wikipedia.org/wiki/Lewis_Hamilton').text

# sample = "//upload.wikimedia.org/wikipedia/commons/thumb"

# start = text.find(sample)
# end = text[start:].find('"')

# 'https:' + text[start:(start + end)]

In [10]:
# text = requests.request("GET", pilots[pilots['Name'] == 'Lewis Hamilton']['url'].values[0]).text

# sample = "//upload.wikimedia.org/wikipedia/commons/thumb"

# start = text.find(sample)
# end = text[start:].find('"')

# 'https:' + text[start:(start + end)]

In [11]:
# url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/18/Lewis_Hamilton_2016_Malaysia_2.jpg/640px-Lewis_Hamilton_2016_Malaysia_2.jpg'

# response = requests.get(url)

# Image.open(BytesIO(response.content))

#### Getting driver details

In [12]:
# Requesting data
data = requests.request("GET", 'http://ergast.com/api/f1/drivers/alonso').text

# Parsing to DataFrame
pilots = pd.read_xml(
    data,
    namespaces = {'doc': "http://ergast.com/mrd/1.5"},
    xpath = './/doc:Driver'
    )

#### Getting qualifying details

In [63]:
# Requesting data
quali = requests.request("GET", 'http://ergast.com/api/f1/drivers/alonso/qualifying?limit=1000').text

In [64]:
# Parsing to DataFrame
qualis = pd.read_xml(
    quali,
    namespaces = {'doc': "http://ergast.com/mrd/1.5"},
    xpath = './/doc:Race'
    )

# Parsing to Tree
tree = ET.fromstring(quali)

In [143]:
#RaceTable [0]
    #Race (season, round, url) [0][0]
        #RaceName {racename}  [0][0][0]
        #Circuit (circuitId, url) [0][0][1]
            #CircuitName {circuitname} [0][0][1][0]
            #Location (lat, long) [0][0][1][1]
                #Locality {Locality} [0][0][1][1][0]
                #Country {Country} [0][0][1][1][1]
        #Date {date} [0][0][2]
        #QualifyingList [0][0][3]
            #QualifyingResult (position) [0][0][3][0]
                #Driver [0][0][3][0][0]
                    #PermanentNumber {}[0][0][3][0][0][0]
                    #GivenName [0][0][3][0][0][1]
                    #FamilyName [0][0][3][0][0][2]
                    #DateOfBirth [0][0][3][0][0][3]
                    #Nationality [0][0][3][0][0][4]
                #Constructor [0][0][3][0][1]
                    #Name [0][0][3][0][1][0]
                    #Nationality

races = len(tree[0])

poles = 0
constructors = {}

nationality = tree[0][0].find('{http://ergast.com/mrd/1.5}QualifyingList').find('{http://ergast.com/mrd/1.5}QualifyingResult').find('{http://ergast.com/mrd/1.5}Driver').find('{http://ergast.com/mrd/1.5}Nationality').text
last_team = tree[0][races-1].find('{http://ergast.com/mrd/1.5}QualifyingList').find('{http://ergast.com/mrd/1.5}QualifyingResult').find('{http://ergast.com/mrd/1.5}Constructor').find('{http://ergast.com/mrd/1.5}Name').text

for i in range(races):
    if (
        tree[0][i]
            .find('{http://ergast.com/mrd/1.5}QualifyingList')
            .find('{http://ergast.com/mrd/1.5}QualifyingResult')
            .attrib['position'] == '1'):
        poles+=1

    constructor = tree[0][i].find('{http://ergast.com/mrd/1.5}QualifyingList').find('{http://ergast.com/mrd/1.5}QualifyingResult').find('{http://ergast.com/mrd/1.5}Constructor').find('{http://ergast.com/mrd/1.5}Name').text
    constructors[constructor] = None

print(nationality)
print(poles)
print(list(constructors.keys()))
print(last_team)
print(races)
print()

Spanish
23
['Minardi', 'Renault', 'McLaren', 'Ferrari', 'Alpine F1 Team']
Alpine F1 Team
326
True


#### Getting races details

In [154]:
# Requesting data
races_data = requests.request("GET", 'http://ergast.com/api/f1/drivers/alonso/results?limit=1000').text

In [155]:
# Parsing to Tree
tree = ET.fromstring(races_data)

In [156]:
races = len(tree[0])
wins = 0
podium = 0

for i in range(races):
    position = int(tree[0][i].find('{http://ergast.com/mrd/1.5}ResultsList').find('{http://ergast.com/mrd/1.5}Result').attrib['position'])
    wins = (wins + 1) if position == 1 else wins
    podium = (podium + 1) if position <= 3 else podium

print(races)
print(wins)
print(podium)

343
32
98


#### Getting championship details

In [179]:
# Requesting data
champioship_data = requests.request("GET", 'https://ergast.com/api/f1/drivers/hamilton/driverStandings/1').text

In [180]:
# Parsing to Tree
tree = ET.fromstring(champioship_data)

In [181]:
len(tree[0])

7

In [184]:
champioship_data = requests.request("GET", f'https://ergast.com/api/f1/drivers/senna/driverStandings').text

In [185]:
# Parsing to Tree
tree = ET.fromstring(champioship_data)

In [192]:
tree[0][0].attrib['season']

'1984'

In [201]:
results = []
wins = 0
for i in range(len(tree[0])):
    season = tree[0][i].attrib['season']
    position = tree[0][i].find('{http://ergast.com/mrd/1.5}DriverStanding').attrib['position'] 
    points = tree[0][i].find('{http://ergast.com/mrd/1.5}DriverStanding').attrib['points']
    emoji = '🥇' if position == '1' else '🥈' if position == '2' else '🥉' if position == '3' else ' '
    wins = wins+1 if position == '1' else wins
    results.append([season, position, points, emoji])

print(wins)
pd.DataFrame(results, columns=['season', 'position', 'points', 'medal'])

3


,season,position,points,medal
0,1984,9,13,
1,1985,4,38,
2,1986,4,55,
3,1987,3,57,🥉
4,1988,1,90,🥇
5,1989,2,60,🥈
6,1990,1,78,🥇
7,1991,1,96,🥇
8,1992,4,50,
9,1993,2,73,🥈


In [206]:
actual = requests.request("GET", f'https://ergast.com/api/f1/current/drivers/alonso/driverStandings').text
tree = ET.fromstring(actual)
position = tree[0][0][0].attrib['position']
points = tree[0][0][0].attrib['points']
wins = tree[0][0][0].attrib['wins']


#### Requsting all circuits

In [69]:
data = requests.request("GET", 'http://ergast.com/api/f1/circuits?limit=10000').text
tree = ET.fromstring(data)

In [70]:
circuits = []
for i in range(len(tree[0])):
    circuitId = tree[0][i].attrib['circuitId']
    circuit = tree[0][i].find('{http://ergast.com/mrd/1.5}CircuitName').text
    geo = tree[0][i].find('{http://ergast.com/mrd/1.5}Location').attrib
    country = tree[0][i].find('{http://ergast.com/mrd/1.5}Location').find('{http://ergast.com/mrd/1.5}Country').text
    circuits.append([circuitId, circuit, geo, country])

pd.DataFrame(circuits, columns=['ID', 'Name', 'Geo', 'Country'])

,ID,Name,Geo,Country
0,adelaide,Adelaide Street Circuit,"{'lat': '-34.9272', 'long': '138.617'}",Australia
1,ain-diab,Ain Diab,"{'lat': '33.5786', 'long': '-7.6875'}",Morocco
2,aintree,Aintree,"{'lat': '53.4769', 'long': '-2.94056'}",UK
3,albert_park,Albert Park Grand Prix Circuit,"{'lat': '-37.8497', 'long': '144.968'}",Australia
4,americas,Circuit of the Americas,"{'lat': '30.1328', 'long': '-97.6411'}",USA
...,...,...,...,...
74,yas_marina,Yas Marina Circuit,"{'lat': '24.4672', 'long': '54.6031'}",UAE
75,yeongam,Korean International Circuit,"{'lat': '34.7333', 'long': '126.417'}",Korea
76,zandvoort,Circuit Park Zandvoort,"{'lat': '52.3888', 'long': '4.54092'}",Netherlands
77,zeltweg,Zeltweg,"{'lat': '47.2039', 'long': '14.7478'}",Austria


In [55]:
print(float(circuits_dataframe[circuits_dataframe['Name'] == 'Ain Diab']['Geo'].values[0]['lat']))
print(float(circuits_dataframe[circuits_dataframe['Name'] == 'Ain Diab']['Geo'].values[0]['long']))

33.5786
-7.6875


In [40]:
list(circuits_dataframe[circuits_dataframe['Country'] == 'Brazil']['Name'])

['Autódromo José Carlos Pace', 'Autódromo Internacional Nelson Piquet']

#### Requesting circuit detail

In [113]:
data = requests.request("GET", 'http://ergast.com/api/f1/1962/circuits/aintree/results?limit=1000').text

tree = ET.fromstring(data)

In [106]:
tree[0][0].find('{http://ergast.com/mrd/1.5}ResultsList').findall('{http://ergast.com/mrd/1.5}Result')

[<Element '{http://ergast.com/mrd/1.5}Result' at 0x00000252719318B0>,
 <Element '{http://ergast.com/mrd/1.5}Result' at 0x0000025271931360>,
 <Element '{http://ergast.com/mrd/1.5}Result' at 0x0000025271933040>,
 <Element '{http://ergast.com/mrd/1.5}Result' at 0x00000252719334A0>,
 <Element '{http://ergast.com/mrd/1.5}Result' at 0x0000025271933A40>,
 <Element '{http://ergast.com/mrd/1.5}Result' at 0x0000025271933F90>,
 <Element '{http://ergast.com/mrd/1.5}Result' at 0x00000252719372C0>,
 <Element '{http://ergast.com/mrd/1.5}Result' at 0x0000025271937900>,
 <Element '{http://ergast.com/mrd/1.5}Result' at 0x0000025271937C70>,
 <Element '{http://ergast.com/mrd/1.5}Result' at 0x000002527192C950>,
 <Element '{http://ergast.com/mrd/1.5}Result' at 0x00000252719144F0>,
 <Element '{http://ergast.com/mrd/1.5}Result' at 0x0000025271914B80>,
 <Element '{http://ergast.com/mrd/1.5}Result' at 0x00000252719182C0>,
 <Element '{http://ergast.com/mrd/1.5}Result' at 0x0000025271918810>,
 <Element '{http://e

In [114]:
details = []
for i in tree[0][0].find('{http://ergast.com/mrd/1.5}ResultsList').findall('{http://ergast.com/mrd/1.5}Result'):
    position = i.attrib['position']
    number = i.attrib['number']
    name = i.find('{http://ergast.com/mrd/1.5}Driver').find('{http://ergast.com/mrd/1.5}GivenName').text
    last_name = i.find('{http://ergast.com/mrd/1.5}Driver').find('{http://ergast.com/mrd/1.5}FamilyName').text
    constructor = i.find('{http://ergast.com/mrd/1.5}Constructor').find('{http://ergast.com/mrd/1.5}Name').text
    emoji = '🥇' if position == '1' else '🥈' if position == '2' else '🥉' if position == '3' else ' '
    details.append([position, number, (name + ' ' + last_name), constructor, emoji])

pd.DataFrame(details)

,0,1,2,3,4
0,1,20,Jim Clark,Lotus-Climax,🥇
1,2,24,John Surtees,Lola,🥈
2,3,16,Bruce McLaren,Cooper-Climax,🥉
3,4,12,Graham Hill,BRM,
4,5,30,Jack Brabham,Lotus-Climax,
5,6,18,Tony Maggs,Cooper-Climax,
6,7,34,Masten Gregory,Lotus-Climax,
7,8,22,Trevor Taylor,Lotus-Climax,
8,9,8,Dan Gurney,Porsche,
9,10,42,Jackie Lewis,Cooper-Climax,
